# Future sales predictor - Kaggle

In [1]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('sales_train.csv')
train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [4]:
train.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [5]:
train.shape

(2935849, 6)

In [26]:
items = pd.read_csv('items.csv')
del items['item_name']
items.head(10)

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40
5,5,40
6,6,40
7,7,40
8,8,40
9,9,40


In [9]:
test = pd.read_csv('test.csv')
test.head(10)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
5,5,5,5039
6,6,5,5041
7,7,5,5046
8,8,5,5319
9,9,5,5003


In [71]:
def clean_data(df):
    data = df
    target = []
    if 'date' in data:
        del data['date']
    if 'date_block_num' in data:
        del data['date_block_num']
    
    if 'item_cnt_day' in data:
        target = data['item_cnt_day']
        del data['item_cnt_day']
    if 'ID' in data:
        target = data['ID']
        prices = pd.DataFrame(train, columns=['item_price, item_id'])
        print(prices)
        data =  data.drop['ID']
        data = pd.merge(data, prices, on='item_id', how='inner')
    
    data = pd.merge(data, items, on='item_id', how='inner')
    
    return data, target

In [72]:
X, y = clean_data(train)

In [73]:
X

,shop_id,item_id,item_price,item_category_id
0,59,22154,999.0,37
1,24,22154,999.0,37
2,27,22154,999.0,37
3,25,22154,999.0,37
4,25,22154,999.0,37
...,...,...,...,...
2935844,25,8428,249.0,40
2935845,25,7903,12198.0,15
2935846,25,7610,2890.0,64
2935847,25,7635,2100.0,64


In [74]:
y

[]

In [76]:
X_real_test, idlist = clean_data(test)
# X_real_test.head(10)

[]

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [2935849, 0]